In [4]:
def readData(dataPath):
    dataFile = open(dataPath)
    dataStr = dataFile.read()
    dataList = dataStr.splitlines()
    return dataList

ordersPath = '/data/retail_db/orders/part-00000'
orders = readData(ordersPath)

orders[:10]



['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE',
 '6,2013-07-25 00:00:00.0,7130,COMPLETE',
 '7,2013-07-25 00:00:00.0,4530,COMPLETE',
 '8,2013-07-25 00:00:00.0,2911,PROCESSING',
 '9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT',
 '10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT']

In [12]:
def getOrderStatusCount(orders):
    StatusCount = {}
    for order in orders:
        orderAttribute = order.split(',')
        if StatusCount.get(orderAttribute[3]):
            StatusCount[orderAttribute[3]] += 1
        else:
            StatusCount[orderAttribute[3]] = 1
    return StatusCount

StatusCount = getOrderStatusCount(orders)

StatusCount
        

{'CLOSED': 7556,
 'PENDING_PAYMENT': 15030,
 'COMPLETE': 22899,
 'PROCESSING': 8275,
 'PAYMENT_REVIEW': 729,
 'PENDING': 7610,
 'ON_HOLD': 3798,
 'CANCELED': 1428,
 'SUSPECTED_FRAUD': 1558}